In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import spacy
from PIL import Image
from wordcloud import WordCloud
import nltk

'''Variables qui seront créées:'''

nlp = spacy.load("en_core_web_sm")
col = ["Auteur",'Auteur_number', "Texte", "File_Name",'Tokens','Tokens_NoStopW','lemma','PartOfSpeech','ent']
df=pd.DataFrame(columns=col)
chemin= os.getcwd() + '\\training_data\\'

In [2]:
def to_Dataframe(chemin):
    '''Cette fonction va ajouter les fichiers txt au dataframe df et va retourner la df '''
    datas = []
    file_dir = []
    file_name = []
    chemin=str(chemin)
    for file in os.listdir(chemin):
        file_name.append(file)
        file_dir= chemin+str(file)
        with open(file_dir, 'r') as f:
            data = f.read()
            datas.append(data)
            f.close()
    df["Texte"]=datas
    df["File_Name"]=file_name
    pattern= "[A-Z]{3}[.]+txt$"
    search = []
    for values in df["File_Name"]:
        search.append(re.search(r'[A-Z]{3}', values).group())
    df['Auteur'] = search
    #df = pd.get_dummies(df, columns=['Auteur'])
    df['Auteur_number']=df['Auteur'].map({'EAP': 1, 'HPL':2, 'MWS':3})
    return df

In [3]:
to_Dataframe(chemin)
print(df)


     Auteur  Auteur_number                                              Texte  \
0       MWS              3  Idris was well content with this resolve of mi...   
1       HPL              2  I was faint, even fainter than the hateful mod...   
2       EAP              1  Above all, I burn to know the incidents of you...   
3       EAP              1  He might see, perhaps, one or two points with ...   
4       MWS              3  All obeyed the Lord Protector of dying England...   
...     ...            ...                                                ...   
3595    MWS              3  My countenance was painted with the hues of il...   
3596    HPL              2  It must have been merely the association of an...   
3597    HPL              2  Dangers he met unflinchingly; crimes he commit...   
3598    MWS              3  As this state of wretchedness became more conf...   
3599    MWS              3  Were the pride of ancestry, the patrician spir...   

               File_Name To

In [4]:
def tokenization(Texte):
    '''Fonction pour tokeniser le texte. Il faut donner comme paramètre la colonne où se trouve le texte.
    Ici: df["Texte]'''
    tokens = []
    lemma = []
    pos = []
    stop_words = []
    ent = []
    for doc in nlp.pipe(Texte.astype('unicode').values, batch_size=50, n_threads=3):
        if doc.is_parsed:
            tokens.append([n.text for n in doc])
            lemma.append([n.lemma_ for n in doc])
            pos.append([n.pos_ for n in doc])
            stop_words.append([n.text for n in doc if not n.is_stop])
            ent.append([e.label_ for e in doc.ents])
        else:
            # We want to make sure that the lists of parsed results have the
            # same number of entries of the original Dataframe, so add some blanks in case the parse fails
            tokens.append(None)
            lemma.append(None)
            pos.append(None)
            stop_words.append(None)
            ent.append(None)
    df['Tokens'] = tokens
    df['Tokens_NoStopW'] = stop_words
    df['lemma'] = lemma
    df['PartOfSpeech'] = pos
    df['ent'] = ent
'''
def final_clean(df):
    '''Nettoyage de la colonne lemma de la lemmatization'''
    df['clean_text']=str()
    df['clean_text']=''.join(df['lemma'])
    df['clean_text']=str(df['clean_text'])
    return df

SyntaxError: invalid syntax (<ipython-input-4-fe8fec62be56>, line 31)

In [ ]:
tokenization(df["Texte"])

In [ ]:
final_clean(df)

In [ ]:
df_finale=df[['Auteur_number','clean_text','ent']]


In [ ]:
def WordCloud_texte():
    texte_HPL = []
    texte_EAP = []
    texte_MWS = []
    for i, row in df.iterrows():
        if row["Auteur"] == 'EAP':
            texte_EAP.append(row["Tokens_NoStopW"])
        if row["Auteur"] == 'HPL':
            texte_HPL.append(row["Tokens_NoStopW"])
        if row["Auteur"] == 'MWS':
            texte_MWS.append(row["Tokens_NoStopW"])
    texte_EAP = ''.join(str(v) for v in texte_EAP)
    texte_HPL = ''.join(str(v) for v in texte_HPL)
    texte_MWS = ''.join(str(v) for v in texte_MWS)

    wordcloud_texte_EAP = WordCloud().generate(texte_EAP)
    plt.imshow(wordcloud_texte_EAP, interpolation='bilinear')
    plt.axis("off")
    plt.show()

    wordcloud_texte_HPL = WordCloud().generate(texte_HPL)
    plt.imshow(wordcloud_texte_HPL, interpolation='bilinear')
    plt.axis("off")
    plt.show()

    wordcloud_texte_MWS = WordCloud().generate(texte_MWS)
    plt.imshow(wordcloud_texte_MWS, interpolation='bilinear')
    plt.axis("off")
    plt.show()

def WorldCloud_ent():
    ENT_HPL = []
    ENT_EAP = []
    ENT_MWS = []
    for i, row in df.iterrows():
        if row["Auteur"] == 'EAP':
            ENT_EAP.append(row["ent"])
        if row["Auteur"] == 'HPL':
            ENT_HPL.append(row["ent"])
        if row["Auteur"] == 'MWS':
            ENT_MWS.append(row["ent"])
    ENT_EAP = ''.join(str(v) for v in ENT_EAP)
    ENT_HPL = ''.join(str(v) for v in ENT_HPL)
    ENT_MWS = ''.join(str(v) for v in ENT_MWS)

    wordcloud_ENT_EAP = WordCloud().generate(ENT_EAP)
    plt.imshow(wordcloud_ENT_EAP, interpolation='bilinear')
    plt.axis("off")
    plt.show()

    wordcloud_ENT_HPL = WordCloud().generate(ENT_HPL)
    plt.imshow(wordcloud_ENT_HPL, interpolation='bilinear')
    plt.axis("off")
    plt.show()

    wordcloud_ENT_MWS = WordCloud().generate(ENT_MWS)
    plt.imshow(wordcloud_ENT_MWS, interpolation='bilinear')
    plt.axis("off")
    plt.show()

def clean_token():
    df['clean_text'] = str()
    for i, row in df.iterrows():
        row['clean_text'] = ' '.join(row['lemma'])
    df['clean_text'] = df.clean_text.replace("[PRON\s\W]", " ", regex=True)
    df['clean_text'] = df.clean_text.replace(' +', ' ', regex=True)
    df['clean_text'] = df.clean_text.replace('^ ', '', regex=True)


def auteur_map():
    df['Auteur_number'] = df['Auteur'].map({'EAP': 1, 'HPL': 2, 'MWS': 3})

df_finale=df[['Auteur_number','clean_text','ent']]


'''